In [13]:
from PIL import Image, ImageOps
import os

def find_max_dimensions(image_paths):
    """
    Finds the maximum width and height among a list of images.
    """
    max_width = 0
    max_height = 0
    for path in image_paths:
        with Image.open(path) as img:
            max_width = max(max_width, img.width)
            max_height = max(max_height, img.height)
    return max_width, max_height

def create_fade_frames(image1, image2, steps):
    """
    Creates frames for fading from image1 to image2.

    :param image1: The starting image.
    :param image2: The ending image.
    :param steps: Number of steps (frames) in the fade.
    :return: List of PIL Image objects.
    """
    fade_frames = []
    for step in range(steps):
        alpha = step / float(steps - 1)  # Calculate blend factor
        blended = Image.blend(image1, image2, alpha)
        fade_frames.append(blended)
    return fade_frames

def create_gif_with_fade(image_folder, output_path, fade_duration, display_duration, fade_steps=10):
    """
    Creates a gif from a series of images with fade effect and display time.

    :param image_folder: The folder where the images are stored.
    :param output_path: The path to save the GIF.
    :param fade_duration: Duration of the fade between images in milliseconds.
    :param display_duration: Duration of displaying each image before fading.
    :param fade_steps: Number of steps in the fade.
    """
    image_paths = [os.path.join(image_folder, f"{i}.png") for i in range(1, 7)]
    max_width, max_height = find_max_dimensions(image_paths)

    processed_images = []
    for path in image_paths:
        with Image.open(path) as img:
            if img.mode != 'RGBA':
                img = img.convert('RGBA')
            img = ImageOps.pad(img, size=(max_width, max_height), centering=(0.5, 0.5))
            processed_images.append(img)

    frames = []
    for i in range(len(processed_images) - 1):
        # Add the image itself for display
        frames.extend([processed_images[i]] * int(display_duration / 100))  # Assuming 100ms per frame
        # Add fade frames
        frames.extend(create_fade_frames(processed_images[i], processed_images[i+1], fade_steps))

    # Add the last image for display
    frames.extend([processed_images[-1]] * int(display_duration / 100))

    # Save the GIF
    # Duration for each frame is set to 100ms
    frames[0].save(output_path, format='GIF', append_images=frames[1:], save_all=True, duration=100, loop=0)

# Example usage
create_gif_with_fade('img', 'output.gif', 50, 1000, 3)
